In [1]:
# Importing neccessary libraries
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import string
from argparse import Namespace
from collections import Counter
import re

In [2]:
# Setting training device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [3]:
special_char = re.compile(r'[;\\/,!.:*?\"<>|&\']')

In [4]:
resp = re.sub(special_char, " ", "Hey,,,,Holla,.!")
print(resp)

Hey    Holla   


In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("hamspam.csv", index_col = ["Unnamed: 0"])
df.head()

,v1,v2,split
0,ham,"Go until jurong point, crazy.. Available only ...",train
1,ham,Ok lar... Joking wif u oni...,train
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,train
3,ham,U dun say so early hor... U c already then say...,train
4,ham,"Nah I don't think he goes to usf, he lives aro...",train


In [7]:
df = df.rename({"v2" : "message"}, axis = "columns")

In [8]:
token_to_idx = {}
idx_to_token = {}

In [11]:
def OneHotEncoder(df, df_column):
    uniq_values = df[df_column].unique()
    number_of_features = len(uniq_values)
    main_dict = {}
    zero_list = np.zeros(len(df))
    for num, val in enumerate(uniq_values):
        # If you have main_dict[val] = zero_list, then you invariably create a reference and this would mess up the One Hot Encoding
        main_dict[val] = list(zero_list)
    for num, record in enumerate(df[df_column]):
        main_dict[str(record)][num] = 1.0
    df_one_hot = pd.DataFrame.from_dict(main_dict)
    frames = [df, df_one_hot]
    df_result = pd.concat(frames,axis=1, join='inner')
    return df_result

In [12]:
df = OneHotEncoder(df, "v1")
print(df.head())

     v1                                            message  split  ham  spam
0   ham  Go until jurong point, crazy.. Available only ...  train  1.0   0.0
1   ham                      Ok lar... Joking wif u oni...  train  1.0   0.0
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...  train  0.0   1.0
3   ham  U dun say so early hor... U c already then say...  train  1.0   0.0
4   ham  Nah I don't think he goes to usf, he lives aro...  train  1.0   0.0


In [13]:
df.drop(["v1"], axis = 1, inplace = True)

In [14]:
train_df = df[df["split"] == "train"]
val_df = df[df["split"] == "val"]
test_df = df[df["split"] == "test"]

In [15]:
train_df.drop(["split"], axis = 1 ,inplace=True)

/var/folders/qp/h29sqr1j4qlg8r6r10m9q9xh0000gn/T/ipykernel_54181/539089241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(["split"], axis = 1 ,inplace=True)


In [16]:
train_df.head()

,message,ham,spam
0,"Go until jurong point, crazy.. Available only ...",1.0,0.0
1,Ok lar... Joking wif u oni...,1.0,0.0
2,Free entry in 2 a wkly comp to win FA Cup fina...,0.0,1.0
3,U dun say so early hor... U c already then say...,1.0,0.0
4,"Nah I don't think he goes to usf, he lives aro...",1.0,0.0


In [17]:
target = torch.tensor(np.stack(train_df[["ham", "spam"]].values.tolist()))

In [18]:
a_target = []

for index, rows in train_df.iterrows():
    if rows["ham"] == 1.0:
        a_target.append([0])
    else:
        a_target.append([1])

In [19]:
a_target = torch.Tensor(a_target).type(torch.LongTensor)
a_target

tensor([[0],
        [0],
        [1],
        ...,
        [0],
        [0],
        [0]])

In [47]:
a_target = a_target.flatten()
a_target

tensor([0, 0, 1,  ..., 0, 0, 0])

In [49]:
a_target.size()

torch.Size([3900])

In [48]:
# Don't run this, I am checking size
output_vector = torch.zeros(len(train_df))
output_vector.size()

torch.Size([3900])

In [20]:
target.shape

torch.Size([3900, 2])

In [21]:
def add_token(token):
    if token in token_to_idx:
        index = token_to_idx[token]
    else:
        index = len(token_to_idx)
        token_to_idx[token] = index
        idx_to_token[index] = token

In [22]:
for sentences in df.message:
    for word in sentences.split(" "):
        word = re.sub(special_char, " ", word)
        word = word.lower()
        add_token(word)


In [23]:
add_token("<UNK>")

In [24]:
def CustomVectorizer(message):
    one_hot = torch.zeros(len(message), 1, len(token_to_idx))
    for num, word in enumerate(message.split(" ")):
        word = re.sub(special_char, " ", word)
        word = word.lower()
        if word in token_to_idx:
            one_hot[num][0][token_to_idx[word]] = 1
        else:
            word = "<UNK>"
            one_hot[num][0][token_to_idx[word]] = 1
    
    return one_hot

In [25]:
for sentence in df["message"]:
    print(CustomVectorizer(sentence).size())
    break

torch.Size([111, 1, 12461])


In [26]:
token_to_idx["<UNK>"]

12460

In [27]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size ,output_size)
        self.softmax = nn.LogSoftmax(dim = 1)
        
    def forward(self, input_message, hidden):
        combined = torch.cat((input_message, hidden), 1)
        hidden = self.i2h(combined)
        output = self.h2o(hidden)
        output = self.softmax(output)
        
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
        

In [28]:
input_size = len(token_to_idx)
hidden_size = len(token_to_idx)
output_size = 2

In [29]:
input_text = train_df["message"][0]
print(len(input_text))

111


In [30]:
rnn = RNN(input_size, hidden_size, output_size)

In [31]:
input_text = CustomVectorizer(train_df["message"][0])
hidden = torch.zeros(1, hidden_size)
output, next_hidden = rnn(input_text[0], hidden)
print(output)

tensor([[-0.6891, -0.6973]], grad_fn=<LogSoftmaxBackward0>)


In [32]:
print(input_text.shape)

torch.Size([111, 1, 12461])


In [33]:
def classifyFromOutput(output):
    top_n, top_i = output.topk(1)
    class_i = top_i[0].item()
    if class_i == 1:
        return "Spam"
    else:
        return "Ham"

In [34]:
classifyFromOutput(output)

'Ham'

In [35]:
criterion = nn.NLLLoss()

In [68]:
subset_df = train_df[:5]
subset_target = a_target[:5]

In [97]:
learning_rate = 0.005
def train(message_df, target):
    hidden = rnn.initHidden()
    range_df = len(message_df)
    rnn.zero_grad()
    output_vector  = torch.empty((0, 2), dtype = torch.float32)
    for message_num in tqdm(range(range_df)):
        vectorized_input = CustomVectorizer(message_df["message"][message_num])
        # Individual input processed by RNN
        for i in range(vectorized_input.size()[0]):
            output, hidden = rnn(vectorized_input[i],hidden)
        output_vector = torch.cat((output_vector, output), 0)
        
    loss = criterion(output_vector, target)
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
        
    return loss.item()

In [98]:
n_iters = 1

for iteration in range(n_iters):
    current_loss = 0
    print("Iteration Number :",iteration)
    loss = train(train_df, a_target)
    current_loss += loss
    print("Loss ", current_loss)

Iteration Number : 0


  3%|██▎                                                                          | 115/3900 [05:52<3:13:07,  3.06s/it]


KeyboardInterrupt: 

In [38]:
import time
import math

n_iters = 10

current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

In [41]:
for iter in range(1, n_iters + 1):
    print("Itertion Number: ", iter)
    current_loss = 0
    for num, input_text in enumerate(train_df["message"]):
        print("Message Number: ", num)
        target_value = a_target[num]
        # This will work -> target_value = torch.tensor([1])
        vectorized_input = CustomVectorizer(input_text)
        output, loss = train(vectorized_input, target_value)
        current_loss += loss
    print("Loss ", current_loss)

Itertion Number:  1
Message Number:  0
Output tensor([[-0.6777, -0.7089]], grad_fn=<LogSoftmaxBackward0>)
Target tensor([0])
Loss tensor(0.6777, grad_fn=<NllLossBackward0>)
Message Number:  1
Output tensor([[-0.6736, -0.7131]], grad_fn=<LogSoftmaxBackward0>)
Target tensor([0])
Loss tensor(0.6736, grad_fn=<NllLossBackward0>)


KeyboardInterrupt: 

In [102]:
input_text = CustomVectorizer("hel, me, ewas")
hidden = torch.zeros(1, hidden_size)
output, next_hidden = rnn(input_text[0], hidden)
print(output)

tensor([[-0.6727, -0.7140]], grad_fn=<LogSoftmaxBackward0>)


In [103]:
for i in range(len(input_text)):
    output, hidden = rnn(input_text[i],hidden)

In [104]:
torch.argmax(output)

tensor(0)

In [118]:
# What if we put vectorized input and ham spam value as a Dataframe
subset_df = pd.DataFrame(list(zip(vectorized_input, a_target)), columns = ["message", "target"])
subset_df.head()

,message,target
0,"[[[tensor(1.), tensor(0.), tensor(0.), tensor(...",tensor(0)
1,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(...",tensor(0)
2,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(...",tensor(1)
3,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(...",tensor(0)
4,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(...",tensor(0)


In [119]:
features = subset_df["message"]
labels = subset_df["target"]

In [124]:
features.shape

(3900,)

In [117]:
# We did not batch anything, do that
vectorized_input = []
print(vectorized_input)
for message in tqdm(train_df["message"]):
#     print(CustomVectorizer(message))
#     vectorized_input = torch.cat((vectorized_input, CustomVectorizer(message)), 0)
    vectorized_input.append(CustomVectorizer(message))

[]


100%|████████████████████████████████████████████████████████████████████████████| 3900/3900 [00:02<00:00, 1893.14it/s]


In [110]:
vectorized_input.shape

AttributeError: 'list' object has no attribute 'shape'

In [106]:
learning_rate = 0.005
def train(message_df, target):
    hidden = rnn.initHidden()
    range_df = len(message_df)
    rnn.zero_grad()
    output_vector  = torch.empty((0, 2), dtype = torch.float32)
    for message_num in tqdm(range(range_df)):
        vectorized_input = message_df[message_num]
        # Individual input processed by RNN
        for i in range(vectorized_input.size()[0]):
            output, hidden = rnn(vectorized_input[i],hidden)
        output_vector = torch.cat((output_vector, output), 0)
        
    loss = criterion(output_vector, target)
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
        
    return loss.item()

In [108]:
n_iters = 1

for iteration in range(n_iters):
    current_loss = 0
    print("Iteration Number :",iteration)
    loss = train(vectorized_input, a_target)
    current_loss += loss
    print("Loss ", current_loss)

Iteration Number : 0


  0%|▏                                                                             | 12/3900 [00:50<4:30:12,  4.17s/it]


KeyboardInterrupt: 

In [120]:
def train(epochs):
    x_train = Variable(torch.from_numpy(features)).float()
    y_train = Variable(torch.from_numpy(labels)).long()
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = model(x_train)
        loss = criterion(y_pred, y_train)
        print ("epoch #",epoch)
        print ("loss: ", loss.item())
        pred = torch.max(y_pred, 1)[1].eq(y_train).sum()
        print ("acc:(%) ", 100*pred/len(x_train))
        loss.backward()
        optimizer.step()

In [122]:
from torch.autograd import Variable

In [123]:
train(100)

TypeError: expected np.ndarray (got Series)